# Exercise 1.04: Tic-Tac-Toe Static Evaluation with a Heuristic Function
In this exercise, you will be performing a static evaluation on the tic-tac-toe game using a heuristic function.

1.- Reuse the code from Steps 1–5 of Activity 1.01

In [1]:
from random import choice

combo_indices = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8],
    [0, 3, 6],
    [1, 4, 7],
    [2, 5, 8],
    [0, 4, 8],
    [2, 4, 6]
]

signo_vacio = '.'
signo_ia = 'X'
signo_oponente = 'O'

def print_board(board):
    print(" ")
    print(' '.join(board[:3]))
    print(' '.join(board[3:6]))
    print(' '.join(board[6:]))
    print(" ")

def opponent_move(board, row, column):
    index = 3 * (row - 1) + (column - 1)
    if board[index] == signo_vacio:
        return board[:index] + signo_oponente + board[index+1:]
    return board

def all_moves_from_board(board, sign):
    lista_movimientos = []
    for i, v in enumerate(board):
        if v == signo_vacio:
            lista_movimientos.append(board[:i] + sign + board[i+1:])
    return lista_movimientos

def ai_move(board):
    return choice(all_moves_from_board(board, signo_ia))

def game_won_by(board):
    for indice in combo_indices:
        if board[indice[0]] == board[indice[1]] == board[indice[2]] != signo_vacio:
            return board[indice[0]]
    return signo_vacio

def game_loop():
    board = signo_vacio * 9
    contador_vacias = 9
    juego_terminado = False
    while contador_vacias > 0 and not juego_terminado:
        if contador_vacias % 2 == 1:
            board = ai_move(board)
        else:
            fila = int(input('Ingresa fila: '))
            columna = int(input('Ingresa columna: '))
            board = opponent_move(board, fila, columna)
        print_board(board)
        juego_terminado = game_won_by(board) != signo_vacio
        contador_vacias = sum(1 for cell in board if cell == signo_vacio)
    print('El juego ha finalizado.')

def all_moves_from_board_list(board_list, sign):
    lista_movimientos = []
    for board in board_list:
        lista_movimientos.extend(all_moves_from_board(board, sign))
    return lista_movimientos

def filter_wins(lista_movimientos, ia_gana, oponente_gana):
    for board in lista_movimientos:
        won_by = game_won_by(board)
        if won_by == signo_ia:
            ia_gana.append(board)
            lista_movimientos.remove(board)
        elif won_by == signo_oponente:
            oponente_gana.append(board)
            lista_movimientos.remove(board)

def count_possibilities():
    board = signo_vacio * 9
    lista_movimientos = [board]
    ia_gana = []
    oponente_gana = []
    for i in range(9):
        print('Step ' + str(i) + '. Moves: ' + str(len(lista_movimientos)))
        sign = signo_ia if i % 2 == 0 else signo_oponente
        lista_movimientos = all_moves_from_board_list(lista_movimientos, sign)
        filter_wins(lista_movimientos, ia_gana, oponente_gana)
    print('First player wins: ' + str(len(ia_gana)))
    print('Second player wins: ' + str(len(oponente_gana)))
    print('Draw', str(len(lista_movimientos)))
    print('Total', str(len(ia_gana) + len(oponente_gana) + len(lista_movimientos)))
    return len(ia_gana), len(oponente_gana), len(lista_movimientos), len(ia_gana) + len(oponente_gana) + len(lista_movimientos)

2.- Create a function that takes the board as input and returns $0$ if the cell is empty, and $-1$ if it's not empty

In [2]:
def takes_board_as_input(board):
    return [0 if cell == signo_vacio else -1 for cell in board]

3.- Create a function that takes the utility vector of possible moves, takes three indices inside the utility vector representing a triple, and returns a function.

  > **Hints**  
  > the returned function will expect a `points` parameter and the `utilities` vector as input and will add points to each cell in $(i, j, k)$, as long as the original value of that cell is non-negative $(>=0)$. In other words, we increased the utility of empty cells only.

In [3]:
def takes_unity_vector(utilities, i, j, k):
    def add_score(points):
        if utilities[i] >= 0:
            utilities[i] += points
        if utilities[j] >= 0:
            utilities[j] += points
        if utilities[k] >= 0:
            utilities[k] += points
    return add_score

4.- Now, create the utility matrix belonging to any board constellation where you will add the `generate_add_score` function defined previously to update the score. You will also implement the rules that we discussed prior to this activity. These rules are as follows:
  * Two AI signs in a row, column, or diagonal, and the third cell is empty: +1000 for the empty cell.
  * The opponent has two signs in a row, column, or diagonal, and the third cell is empty: +100 for the empty cell.
  * One AI sign in a row, column, or diagonal, and the other two cells are empty: +10 for the empty cells.
  * No AI or opponent signs in a row, column, or diagonal: +1 for the empty cells.

In [4]:
def utility_matrix(board):
    utilities = takes_board_as_input(board)
    for [i, j, k] in combo_indices:
        add_score = takes_unity_vector(utilities, i, j, k)
        triple = [board[i], board[j], board[k]]
        if triple.count(signo_vacio) == 1:
            if triple.count(signo_ia) == 2:
                add_score(1000)
            elif triple.count(signo_oponente) == 2:
                add_score(100)
        elif triple.count(signo_vacio) == 2 and triple.count(signo_ia) == 1:
            add_score(10)
        elif triple.count(signo_vacio) == 3:
            add_score(1)
    return utilities

5.- Create a function that selects the move with the highest utility value. If multiple moves have the same utility, the function returns both moves

In [5]:
def highest_utility_value(board, sign):
    lista_movimientos = []
    utilities = utility_matrix(board)
    max_utility = max(utilities)
    for i, v in enumerate(board):
        if utilities[i] == max_utility:
            lista_movimientos.append(board[:i] + sign + board[i+1:])
    return lista_movimientos

def all_moves_from_board_list(board_list, sign):
    lista_movimientos = []
    get_moves = highest_utility_value if sign == signo_ia else all_moves_from_board
    for board in board_list:
        lista_movimientos.extend(get_moves(board, sign))
    return lista_movimientos

6.- Run the application.

Output:

```
step 0. Moves: 1
step 1. Moves: 1
step 2. Moves: 8
step 3. Moves: 24
step 4. Moves: 144
step 5. Moves: 83
step 6. Moves: 214
step 7. Moves: 148
step 8. Moves: 172
First player wins: 504
Second player wins: 12
Draw 91
Total 607
```

In [6]:
first_player, second_player, draw, total = count_possibilities()

Step 0. Moves: 1
Step 1. Moves: 1
Step 2. Moves: 8
Step 3. Moves: 24
Step 4. Moves: 144
Step 5. Moves: 83
Step 6. Moves: 214
Step 7. Moves: 148
Step 8. Moves: 172
First player wins: 504
Second player wins: 12
Draw 91
Total 607


By completing this exercise, we have observed that the AI is underperforming compared to our previous activity, Activity 1.03, Fixing the First and Second Moves of the AI to Make It Invincible. In this situation, hardcoding the first two moves was better than setting up the heuristic, but this is because we haven't set up the heuristic properly.